In [171]:
import pandas as pd
import numpy as np
data = pd.read_excel(r'D:\Example_Data.xlsx',sheet_name='Example_Data',header=1)
data_DB = pd.read_excel(r'D:\Example_Data.xlsx',sheet_name='Example_DB')

In [173]:
#数据清洗
drop_row = {}
for i in data.index:
    flag = 0
    ID = data.loc[i]['Company ID']
    Name = data.loc[i]['Company Name']
    for j in data[data['Company ID'] == ID].index:
        if data.loc[j]['Company Name'] != Name:
            flag = 1
    if flag == 1:
        drop_row[Name] = ID
for i in drop_row.values():
    data = data[data['Company ID'] != i]

In [174]:
years = []
tc = ['USD', 'GBP']
for i in range(23):
    years.append(str(i+1999))
for i in data.index:
    flag = 0
    Year = str(data.loc[i]['Fiscal Year'])
    SC = str(data.loc[i]['SIC Code'])
    TC = data.loc[i]['Trading Currency']
    SP = data.loc[i]['SP']
    CDS = data.loc[i]['CDS']
    APD = data.loc[i]['APD']
    ARD = data.loc[i]['ARD']
    ADA = data.loc[i]['ADA']
    if Year not in years:
        flag = 1
    elif len(SC) != 4:
        flag = 1
    elif TC not in tc:
        flag = 1
    elif ~np.isnan(SP) and (int(SP) != SP):
        flag = 1
    elif ~np.isnan(CDS) and (int(CDS) != CDS):
        flag = 1
    elif ~np.isnan(APD) and (int(APD) != APD):
        flag = 1
    elif ~np.isnan(ARD) and (int(ARD) != ARD):
        flag = 1
    elif ~np.isnan(ADA) and (int(ADA) != ADA):
        flag = 1
    if flag == 1:
        data = data.drop(i, axis = 0)

In [176]:
#数据处理
col = list(data.columns[:-5])
col.append('Metric Name')
col.append('values')
data_new = pd.DataFrame(columns = col)
MN = ['SP','CDS','APD','ARD','ADA']
n = 0
for i in data.index:
    for j in MN:
        data_new.loc[n] = data.loc[i][:-5]
        data_new.loc[n, 'Metric Name'] = j
        data_new.loc[n, 'values'] = data.loc[i][j]
        n += 1

In [196]:
data_new

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,values
0,4993687,A Company,2010,Machinery,6085,USD,SP,75
1,4993687,A Company,2010,Machinery,6085,USD,CDS,564
2,4993687,A Company,2010,Machinery,6085,USD,APD,109
3,4993687,A Company,2010,Machinery,6085,USD,ARD,44
4,4993687,A Company,2010,Machinery,6085,USD,ADA,0
...,...,...,...,...,...,...,...,...
430,5081187,M Company,2018,Machinery,5886,GBP,SP,97
431,5081187,M Company,2018,Machinery,5886,GBP,CDS,NaN
432,5081187,M Company,2018,Machinery,5886,GBP,APD,104
433,5081187,M Company,2018,Machinery,5886,GBP,ARD,121


In [213]:
#数据对比
column = list(data_new.columns[:-1])
Mdata = pd.merge(data_new, data_DB, how='outer', on=column)
for i in Mdata.index:
    FL = Mdata.loc[i]['values']
    DB = Mdata.loc[i]['Value']
    if np.isnan(FL) and np.isnan(DB):
        Mdata.loc[i,'ERROR Type'] = 1
    elif np.isnan(FL) and ~np.isnan(DB):
        Mdata.loc[i,'ERROR Type'] = 'Not_in_file'
    elif ~np.isnan(FL) and np.isnan(DB):
        Mdata.loc[i,'ERROR Type'] = 'Not_in_DB'
    elif FL != DB:
        Mdata.loc[i,'ERROR Type'] = 'UnEqual'
    else:
         Mdata.loc[i,'ERROR Type'] = 1

In [238]:
Mdata[Mdata['ERROR Type']!=1]

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,values,Value,ERROR Type
91,4994275,D Company,2011,Machinery,5722,USD,CDS,NaN,0.0,Not_in_file
96,4994275,D Company,2012,Machinery,5722,USD,CDS,NaN,0.0,Not_in_file
315,4996197,K Company,2015,Machinery,578B,USD,SP,29,NaN,Not_in_DB
317,4996197,K Company,2015,Machinery,578B,USD,APD,101,NaN,Not_in_DB
318,4996197,K Company,2015,Machinery,578B,USD,ARD,102,NaN,Not_in_DB
319,4996197,K Company,2015,Machinery,578B,USD,ADA,0,NaN,Not_in_DB
365,4986826,L Company,2015,Auto Components,6090,USD,SP,179,17.0,UnEqual
375,4986826,L Company,2017,Auto Components,6090,USD,SP,196,19.0,UnEqual
